In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
import folium
from pandas import json_normalize

In [2]:
geo=Nominatim(user_agent='hkjx')

CLIENT_ID="2Y10RMHFDSVKCQNL0FJV5L2LHRFFNV022NAGWHSC5FSY20L4"
CLIENT_SECRET="BGYY1LU0KEVTCCU23FIWRJHIHJWWMXDZPEZRLKPGNAFBD3RY"
VERSION="20200101"

In [3]:
source=requests.get('https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi').text
soup=BeautifulSoup(source,'lxml')
soup=soup.find('div',class_='mw-parser-output')

table=[]
district=[]
for i in soup.find_all('li'):
    table.append(i.text)
    
delhi=pd.DataFrame(table[12:198],columns=['Area'])

In [4]:
delhi

,Area
0,Adarsh Nagar
1,Ashok Vihar
2,Begum Pur
3,Karala
4,Model Town
...,...
181,Mayur Vihar
182,Meera Bagh
183,Mehrauli
184,Model Town


In [5]:
delhi['lat']=np.zeros(delhi.shape[0])
delhi['long']=np.zeros(delhi.shape[0])

In [6]:
for i,area in delhi.iterrows():
    try: 
        address=(area['Neighborhood']+',Delhi')
        print(i,address)
        loc=geo.geocode(address)
        delhi.iloc[i,1]=loc.latitude
        delhi.iloc[i,2]=loc.longitude
    except:
        pass

In [7]:
delhi=delhi[delhi['lat']!=0]

In [8]:
delhi.reset_index(drop=True,inplace=True)

In [9]:
loc=geo.geocode('Delhi')

In [13]:
delhi_map=folium.Map([loc.latitude,loc.longitude],zoom_start=10)
for lat,long,area in zip(delhi['lat'],delhi['long'],delhi['Neighborhood']):
    label=folium.Popup(area,parse_html=True)
    folium.features.CircleMarker([lat,long],popup=label,color='blue',radius=5,fill=True).add_to(delhi_map)
delhi_map

In [14]:
delhi.shape

(170, 5)

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        print('.',end='')
        try:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                100)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
delhi_venues=getNearbyVenues(delhi['Neighborhood'],delhi['lat'],delhi['long'])

..........................................................................................................................................................................

In [24]:
#delhi_venues.to_csv('dehi_venues.csv')
delhi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.714401,77.167288,Vishyavidyalaya Metro Station@Entry gate #1 n ...,28.715596,77.170981,Train Station
1,Adarsh Nagar,28.714401,77.167288,Adarsh Nagar Metro Station,28.716598,77.170436,Light Rail Station
2,Adarsh Nagar,28.714401,77.167288,Pahalwan Dhaba,28.714594,77.172155,Indian Restaurant
3,Adarsh Nagar,28.714401,77.167288,Modern Age Computers,28.716500,77.162900,IT Services
4,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service


In [25]:
delhi_venues.shape

(1183, 7)

In [26]:
venues=delhi_venues['Neighborhood'].unique()

In [28]:
venues

array(['Adarsh Nagar', 'Ashok Vihar', 'Azadpur', 'Bawana', 'Dhaka',
       'Jahangirpuri', 'Keshav Puram', 'Kingsway Camp', 'Kohat Enclave',
       'Model Town', 'Narela', 'Pitam Pura', 'Rani Bagh', 'Rithala',
       'Shalimar Bagh', 'Shakti Nagar', 'Bara Hindu Rao', 'Chandni Chowk',
       'Chawri Bazaar', 'Civil Lines', 'Dariba Kalan', 'Gulabi Bagh',
       'Kamla Nagar', 'Kashmiri Gate', 'Lahori Gate', 'Nehru Vihar',
       'Paharganj', 'Pratap Nagar', 'Pul Bangash', 'Roshanara Bagh',
       'Sadar Bazaar', 'Sant Nagar', 'Sarai Kale Khan', 'Sarai Rohilla',
       'Shastri Nagar', 'Tis Hazari', 'Timarpur', 'Babarpur',
       'Dilshad Garden', 'Kabir Nagar', 'Mandoli', 'Maujpur',
       'Mukherjee Nagar', 'Nand Nagri', 'Naveen Shahdara', 'New Usmanpur',
       'Shahdara', 'Sonia Vihar', 'Shastri Park', 'Daryaganj',
       'Jhandewalan', 'Karol Bagh', 'Barakhamba Road', 'Chanakyapuri',
       'Connaught Place', 'Gautampuri', 'Gole Market', 'Golf Links',
       'INA Colony', 'Khan Marke

In [30]:
delhi_venues.groupby('Neighborhood').count()['Venue']>5


Neighborhood
Adarsh Nagar          False
Alaknanda              True
Anand Vihar           False
Ashok Nagar           False
Ashok Vihar           False
                      ...  
Vasant Vihar           True
Vasundhara Enclave    False
Vikaspuri             False
Vishwas Nagar         False
Vivek Vihar           False
Name: Venue, Length: 152, dtype: bool

In [31]:
unique_cat=delhi_venues['Venue Category'].unique()
restaurant_type=[]
for cat in unique_cat:
    if('Restaurant' in cat):
        restaurant_type.append(cat)
res_table=delhi_venues[delhi_venues['Venue Category'].isin(restaurant_type)]

In [32]:
res_table.reset_index(drop=True, inplace=True)
res_table

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.714401,77.167288,Pahalwan Dhaba,28.714594,77.172155,Indian Restaurant
1,Ashok Vihar,28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant
2,Azadpur,28.707657,77.175547,Tulip Banquet,28.704523,77.172441,Restaurant
3,Kingsway Camp,28.698778,77.204626,Rico's,28.694766,77.204149,Fast Food Restaurant
4,Kingsway Camp,28.698778,77.204626,QD's New,28.695156,77.204081,Chinese Restaurant
...,...,...,...,...,...,...,...
332,Tilak Nagar,28.636548,77.096496,Aggarwal Restaurant,28.636747,77.097177,Indian Restaurant
333,Tilak Nagar,28.636548,77.096496,Bikanerwala,28.633524,77.097492,Fast Food Restaurant
334,Uttam Nagar,28.624847,77.065286,Shree Maakhan,28.625825,77.068608,Indian Restaurant
335,Uttam Nagar,28.624847,77.065286,Appointment,28.625543,77.068737,Indian Restaurant


In [33]:
len(unique_cat)

176

In [47]:
# one hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add area column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move area column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_grouped=delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wings Joint,Women's Store
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
from sklearn.cluster import KMeans 
# set number of clusters
kclusters = 7

delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

delhi_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

delhi_grouped.groupby('Cluster Labels').count()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wings Joint,Women's Store
Cluster Labels,,,,,,,,,,,,,,,,,,,,,
0,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
1,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
2,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18
4,62,62,62,62,62,62,62,62,62,62,...,62,62,62,62,62,62,62,62,62,62
5,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [49]:
delhi_merged = delhi.join(delhi_grouped.set_index('Neighborhood'), on='Neighborhood')

In [51]:
delhi_merged.drop('Unnamed: 0',1,inplace=True)

In [55]:
delhi_merged.dropna(inplace=True)
delhi_merged.head()

,Neighborhood,lat,long,district,Cluster Labels,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wings Joint,Women's Store
0,Adarsh Nagar,28.714401,77.167288,North West,3.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0
1,Ashok Vihar,28.699453,77.184826,North West,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,Azadpur,28.707657,77.175547,North West,4.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,Bawana,28.799660,77.032885,North West,4.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,Dhaka,28.708698,77.205749,North West,4.0,0.0,0.25,0.0,0.0,0.0,...,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
from matplotlib import colors
import matplotlib.cm as cm
import matplotlib.colors as colorsa
# create map
map_clusters = folium.Map(location=[loc.latitude, loc.longitude], zoom_start=11)

# set color scheme for the clusters
colors_array = cm.rainbow(np.linspace(0, 1, 5))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['lat'], delhi_merged['long'], delhi_merged['Neighborhood'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
italian=delhi_merged.groupby('Cluster Labels').mean()['Italian Restaurant']

italian=italian+delhi_merged.groupby('Cluster Labels').mean()['Pizza Place']
italian

Cluster Labels
0.0    0.038675
1.0    0.049904
2.0    0.000000
3.0    0.043254
4.0    0.039815
5.0    0.000000
6.0    0.000000
dtype: float64

In [81]:
cluster1=delhi_merged[delhi_merged['Cluster Labels']==1]

In [86]:
cluster1.sort_values('Italian Restaurant').head()

,Neighborhood,lat,long,district,Cluster Labels,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wings Joint,Women's Store
7,Keshav Puram,28.688926,77.161683,North West,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,Punjabi Bagh,28.668945,77.132461,West,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,Patel Nagar,28.659809,77.156957,West,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,Janakpuri,28.621927,77.087476,West,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,Ashok Nagar,28.636021,77.101822,West,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
